In [28]:
import pandas as pd
import datetime as dt
import urllib
import zipfile
import re
from geopy.distance import vincenty

## download GTFS data

In [4]:
url = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'
urllib.urlretrieve (url, "GTFS_nyc_Subway.zip")

('GTFS_nyc_Subway.zip', <httplib.HTTPMessage instance at 0x10b381560>)

In [5]:
zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", 'r')
zip_ref.extractall("GTFS_nyc_Subway")
zip_ref.close()

# STRUCTURE OF THIS NOTEBOOK
### prepare data for NODES
- get lines at each station
- get sub nodes
- get master nodes

### prepare data for LINKS
- get duration between stops by train
- get transfer between nodes
- get waiting time (link between master_node --> sub_node)


## get lines at each station

In [506]:
stop_times= pd.read_csv('GTFS_nyc_Subway/stop_times.txt')

In [507]:
stop_times.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,A20170625SUN_001150_7..S97R,00:11:30,00:11:30,701S,1,NaN,0,0,NaN
1,A20170625SUN_001150_7..S97R,00:14:00,00:14:00,702S,2,NaN,0,0,NaN


In [508]:
# A20170625SUN_001150_7..S97R
## A20170625SUN means the service by Sunday Schedule
## 001150 is id for the trip, form the 1st stop to last stop, next round would be different id
## 7..S97R means 7 train would run South bound,
## stop at S97R route ( set of stops of local train and express train are different)
stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
del stop_times['stop_headsign']
del stop_times['pickup_type']
del stop_times['drop_off_type']
del stop_times['shape_dist_traveled']

In [510]:
stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])

In [518]:
stop_times.loc[stop_times[stop_times['train+direction'].isin(['N..N72R','N..S72R'])].index,['train']]='W'

In [520]:
stop_times.to_csv('stop_times_cleaned')

In [521]:
stop_train= stop_times[['stop_id','train']].drop_duplicates()

In [522]:
stop_train['stop_id'] = map(lambda x:x[:-1],stop_train['stop_id'])

In [523]:
stop_train = stop_train.drop_duplicates().sort_values(['train','stop_id']).reset_index(drop= True)

In [525]:
set(stop_train.train)

{'1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'FS',
 'G',
 'GS',
 'H',
 'J',
 'L',
 'M',
 'N',
 'Q',
 'R',
 'SI',
 'W'}

No Z train info in schedule data. Z is same as J.

FS(Brooklyn), GS(Time sq- Grand Central), H(JFK) are all S train in map, 

In [11]:
stops=pd.read_csv('GTFS_nyc_Subway/stops.txt')

In [526]:
stop_train = stop_train.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on='stop_id',how='left')

In [527]:
stop_train.head()

,stop_id,train,stop_name,stop_lat,stop_lon
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583
1,103,1,238 St,40.884667,-73.900870
2,104,1,231 St,40.878856,-73.904834
3,106,1,Marble Hill - 225 St,40.874561,-73.909831
4,107,1,215 St,40.869444,-73.915279


In [528]:
stop_train.to_csv('stop_train')

## get sub-nodes

In [529]:
stop_train['node_id']= stop_train['stop_id'] + '_' + stop_train['train']

In [531]:
stop_train.head()

,stop_id,train,stop_name,stop_lat,stop_lon,node_id
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583,101_1
1,103,1,238 St,40.884667,-73.900870,103_1
2,104,1,231 St,40.878856,-73.904834,104_1
3,106,1,Marble Hill - 225 St,40.874561,-73.909831,106_1
4,107,1,215 St,40.869444,-73.915279,107_1


In [535]:
stop_train.to_csv('stop_train')

## get master-nodes

In [533]:
stop_id = stop_train[['stop_id','stop_name','stop_lat','stop_lon']].drop_duplicates()

In [534]:
stop_id.head(2)

,stop_id,stop_name,stop_lat,stop_lon
0,101,Van Cortlandt Park - 242 St,40.889248,-73.898583
1,103,238 St,40.884667,-73.900870


In [538]:
def find_master(stop_name):
    """
    This function is to distinguish which sub-stations belong to same Master Node
    Because some stations have same name, but not belong to same Master node, 
    eg. there are 3 Canal St Master Node, and 6 sub-nodes,
    
    So for two sub nodes of same stop_name,
    if their distance is more than 0.1 mile,
    then these two sub-nodes are under different Master Node
    
    eg.
    if A--B is less than 0.1 mile, A--C less than 0.1 mile, but B--C more than 0.1 mile,
    still consider A,B,C are under same master node
    
    """
    
    
    stop_train =pd.DataFrame.from_csv("stop_train")
    stop_id = stop_train[['stop_id','stop_name','stop_lat','stop_lon']].drop_duplicates()
    
    df = stop_id[stop_id['stop_name']== stop_name]
    dist = []
    for i in range(len(df)):
        lat1 = df.iloc[i,2]
        lon1 = df.iloc[i,3]
        for j in range(i+1,len(df)):
            lat2 = df.iloc[j,2]
            lon2 = df.iloc[j,3]
            dist.append((df.iloc[i,0],df.iloc[j,0],vincenty((lat1,lon1), (lat2,lon2)).miles))
            # dist is a list of distance between all the stop_id,
            # if distance less than 0.1 mile, consider them as the same station
        
    # st is stop_id pair, which distance are under 0.1 mile
    st = map(lambda x :(x[0],x[1]),filter(lambda x : x[2]<0.1, dist))

    #that are under same master node
    li = {}
    num=0
    for i,j in st:
        if i in li.keys():
            if j in li.keys():
                pass
            else:
                li[j]=li[i]
        else:
            li[i] = num
            li[j] = num
            num +=1
    for i in df['stop_id'].values:
        if i in li.keys():
            pass
        else:
            li[i] = num
            num+=1

    df['master_node_id']= None
    for i in range(len(df)):
        stop = df.iloc[i,0]
        df.iloc[i,-1]=df.iloc[i,1] + '_' +str(li[stop])
    return df

In [539]:
master_node = pd.DataFrame(columns=stop_id.columns)
for stop_name in list(set(stop_train['stop_name'])):
    if len(stop_id[stop_id['stop_name']== stop_name])==1:
        df = stop_id[stop_id['stop_name']== stop_name]
        df['master_node_id'] = df['stop_name']+'_'+'0'
        master_node = pd.concat([master_node,df])
    else:
        master_node = pd.concat([master_node,find_master(stop_name)])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [540]:
print 'Sub Nodes Number:', len(master_node['master_node_id'])
print 'Master Nodes Number:', len(set(master_node['master_node_id']))
print 'Unique Station Name:', len(set(master_node['stop_name']))

Sub Nodes Number: 493
Master Nodes Number: 465
Unique Station Name: 373


In [544]:
sub_node = stop_train.merge(master_node[['master_node_id','stop_id']],on='stop_id',how='left')

In [545]:
sub_node.head()

,stop_id,train,stop_name,stop_lat,stop_lon,node_id,master_node_id
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583,101_1,Van Cortlandt Park - 242 St_0
1,103,1,238 St,40.884667,-73.900870,103_1,238 St_0
2,104,1,231 St,40.878856,-73.904834,104_1,231 St_0
3,106,1,Marble Hill - 225 St,40.874561,-73.909831,106_1,Marble Hill - 225 St_0
4,107,1,215 St,40.869444,-73.915279,107_1,215 St_0


In [546]:
sub_node.to_csv('sub_node')

In [547]:
master = master_node.copy()

In [548]:
master = master[['master_node_id','stop_name']].drop_duplicates()

In [549]:
master_node.head()

,master_node_id,stop_id,stop_lat,stop_lon,stop_name
889,St George_0,S31,40.643748,-74.073643,St George
298,Hunts Point Av_0,613,40.820948,-73.890549,Hunts Point Av
874,Huguenot_0,S16,40.533674,-74.191794,Huguenot
649,85 St - Forest Pkwy_0,J16,40.692435,-73.860010,85 St - Forest Pkwy
883,Dongan Hills_0,S25,40.588849,-74.096090,Dongan Hills


In [550]:
li=[]
for i in master['master_node_id'].values:
    li.append(','.join(master_node[master_node['master_node_id']==i]['stop_id'].values))
master['stop_id']  =li

In [551]:
master.sort_values('master_node_id').reset_index(drop = True).to_csv('master_node')

## get duration between stops by train

In [349]:
def find_duration(time1,time2,day='WKD'):
    """
    This funtion is used to get time duration between stops 
    based on different schedule of time of day.
    
    (time1,time2) defines train departure time at first stop between time1 and time 2
    time1 = 0,1,2,...,23
    time2 = 0,1,2,...,23
    eg. time1=7,time2=10, then train departure between 7:00:00 and 9:59:59 count
    day = 'WKD', 'SAT' or 'SUN'
    """
    ## read stop_times data and prepare it\
    stop_times = pd.read_csv('stop_times_cleaned')
    
    ## choose schedule only for weekday or sun, or sat.
    stop_times = stop_times[stop_times['day']==day]
    
    
    ## get all the sub_trip_id departures from the 1st stops between time1 and time2
    morning_rush_trip_id = stop_times[(stop_times['stop_sequence']==1)&
                                   stop_times['arrival_time'].isin(
                                        filter(lambda x: (int(x[:2])>=time1)&
                                       (int(x[:2])<time2),stop_times['arrival_time']))]['sub_trip_id']
    
    ## get all the schedule of all sub_trip 
    morning_rush = stop_times[stop_times['sub_trip_id'].isin(morning_rush_trip_id)]
    morning_rush['arrival_time'] = pd.to_datetime(morning_rush['arrival_time'])
    
    ## get all the route id within the time period
    start = morning_rush[morning_rush['stop_sequence']==1]
    morning_rush_route =  list(set(morning_rush['train+direction'].values))
    
    ## get length of all the route, for next step purpose-- get the sequence of the route
    route_len = {}
    for route in morning_rush_route:
        if route_len.get(route)==None:
            route_len[route]= morning_rush[morning_rush['train+direction'] == route]['stop_sequence'].max()
        else:
            pass
    
#    print route_len
    ## get schedule of all the routes run within the time period
    duration=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration','route'])
    for i in route_len.keys():
        df = morning_rush[morning_rush['train+direction']==i].iloc[:route_len[i]]
        duration_1=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration'])
        duration_1['from_stop_id'] = df['stop_id'][:-1].values
        duration_1['to_stop_id'] = df['stop_id'][1:].values
        duration_1['duration']=map(dt.timedelta.total_seconds,(df['arrival_time'][1:].values-df['arrival_time'][:-1]))
        duration_1['route'] = df['train+direction'].values[:-1]
        duration = pd.concat([duration,duration_1])
        
    ## occasionally, duration between two stops by same train is different from different route
    ## so I use mean to replace all the duplicates
    duration['train'] = map(lambda x:x.split('.')[0],duration['route'])
    duration= duration.reset_index(drop=True)
#    print set(duration['train'])
 
    duration.loc[duration[duration['route'].isin(['N..N72R','N..S72R'])].index,'train']='W'
    duration = pd.DataFrame(duration.groupby(['from_stop_id','to_stop_id','train'])['duration'].mean()).reset_index()
    
    ## read stops data and merge stops name and geo info with duration dataframe
    stops = pd.read_csv('GTFS_nyc_Subway/stops.txt')
    
    duration = duration.merge(stops.rename(columns={'stop_id':'from_stop_id',
                    "stop_name":'from_stop_name',
                     "stop_lat":'from_stop_lat',
                     "stop_lon":'from_stop_lon'})[['from_stop_id','from_stop_name','from_stop_lat','from_stop_lon']],
              on = 'from_stop_id',how = 'left')
    duration = duration.merge(stops.rename(columns={'stop_id':'to_stop_id',
                    "stop_name":'to_stop_name',
                     "stop_lat":'to_stop_lat',
                     "stop_lon":'to_stop_lon'})[['to_stop_id','to_stop_name','to_stop_lat','to_stop_lon']],
              on = 'to_stop_id',how = 'left')
    return duration

In [351]:
duration = find_duration(7,10)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [493]:
duration['from_node'] = map(lambda x: x[:-1],duration ['from_stop_id'])
duration['to_node'] = map(lambda x: x[:-1],duration ['to_stop_id'])
duration['from_node'] = duration['from_node']+ '_' +duration['train']
duration['to_node'] = duration['to_node']+ '_' +duration['train']

In [495]:
duration.head()

,from_stop_id,to_stop_id,train,duration,from_stop_name,from_stop_lat,from_stop_lon,to_stop_name,to_stop_lat,to_stop_lon,from_node,to_node
0,101S,103S,1,90.0,Van Cortlandt Park - 242 St,40.889248,-73.898583,238 St,40.884667,-73.900870,101_1,103_1
1,103N,101N,1,90.0,238 St,40.884667,-73.900870,Van Cortlandt Park - 242 St,40.889248,-73.898583,103_1,101_1
2,103S,104S,1,90.0,238 St,40.884667,-73.900870,231 St,40.878856,-73.904834,103_1,104_1
3,104N,103N,1,90.0,231 St,40.878856,-73.904834,238 St,40.884667,-73.900870,104_1,103_1
4,104S,106S,1,90.0,231 St,40.878856,-73.904834,Marble Hill - 225 St,40.874561,-73.909831,104_1,106_1


In [496]:
duration.to_csv('duration_7-10am_weekday')

## get transfer between nodes

In [366]:
transfer= pd.DataFrame.from_csv('GTFS_nyc_Subway/transfers.txt')
transfer  = transfer.reset_index()
transfer.head(2)

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
0,101,101,2,180
1,103,103,2,180


In [370]:
transfer['from_stop']=None
transfer['to_stop']=None

for i in range(len(transfer)):
    from_id = transfer.iloc[i,0]
    to_id = transfer.iloc[i,1]
    try:
        transfer.iloc[i,4] = stops[stops['stop_id']==from_id]['stop_name'].values[0]
    except IndexError:
        print from_id
        
    try:
        transfer.iloc[i,5] = stops[stops['stop_id']==to_id]['stop_name'].values[0]
    except IndexError:
        print to_id


In [381]:
del transfer['transfer_type']

In [382]:
transfer[transfer['from_stop']!=transfer['to_stop']].head(3)

,from_stop_id,to_stop_id,min_transfer_time,from_stop,to_stop
10,112,A09,180,168 St - Washington Hts,168 St
29,127,A27,300,Times Sq - 42 St,42 St - Port Authority Bus Terminal
37,132,L02,180,14 St,6 Av


In [384]:
transfer = transfer.merge(sub_node[['stop_id','train']].rename(columns={'stop_id':"from_stop_id",
                                                            'train':'from_train'}), on='from_stop_id',how = 'left')
transfer = transfer.merge(sub_node[['stop_id','train']].rename(columns={'stop_id':"to_stop_id",
                                                            'train':'to_train'}), on='to_stop_id',how = 'left')

In [386]:
transfer['from_node_id'] = transfer['from_stop_id']+'_'+transfer['from_train']
transfer['to_node_id'] = transfer['to_stop_id']+'_'+transfer['to_train']

In [388]:
transfer.head()

,from_stop_id,to_stop_id,min_transfer_time,from_stop,to_stop,from_train,to_train,from_node_id,to_node_id
0,101,101,180,Van Cortlandt Park - 242 St,Van Cortlandt Park - 242 St,1,1,101_1,101_1
1,103,103,180,238 St,238 St,1,1,103_1,103_1
2,104,104,180,231 St,231 St,1,1,104_1,104_1
3,106,106,180,Marble Hill - 225 St,Marble Hill - 225 St,1,1,106_1,106_1
4,107,107,180,215 St,215 St,1,1,107_1,107_1


In [500]:
transfer = transfer.dropna(how= 'any')

In [501]:
transfer.to_csv('transfer')

## Links: master_node --> sub_node

In [412]:
stop_times['node_id'] = map(lambda x,y : x+'_'+y,map(lambda x: x[:-1],stop_times['stop_id']),stop_times['train'])

In [467]:
def train_interval(time,node_id,day='WKD',time_delta=1):
    """
    eg. time=7, node_id='R36_R',
    this function is to find all the R train stops at R36 station between 7:00:00 to 7:59:59
    there are 16 R trains stops there.
    Consider North bound and South bound, 2 direction, then divide by 2
    8 R trains in one direction within 1 hour
    return 8
    """
    df= stop_times[(stop_times['day']==day)&(stop_times['node_id'] == node_id)].sort_values('arrival_time')
    df = df[df['arrival_time'].isin(filter(lambda x: (int(x[:2])>=time)&
                                       (int(x[:2])<(time+time_delta)),df['arrival_time']))]
    train_num = len(df)*1.0/2
    return train_num

In [469]:
train_interval(time=7, node_id='R36_R',time_delta=3)

24.5

In [463]:
waiting = sub_node.copy()

In [464]:
waiting.head()

,stop_id,train,stop_name,stop_lat,stop_lon,node_id,master_node_id
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583,101_1,Van Cortlandt Park - 242 St_0
1,103,1,238 St,40.884667,-73.900870,103_1,238 St_0
2,104,1,231 St,40.878856,-73.904834,104_1,231 St_0
3,106,1,Marble Hill - 225 St,40.874561,-73.909831,106_1,Marble Hill - 225 St_0
4,107,1,215 St,40.869444,-73.915279,107_1,215 St_0


In [472]:
# train_num is number of trains in one direction
## here I find train_num for 3 hours, from 7:00 to 10:00
waiting['train_num'] = None
for i in range(len(waiting)):
    node_id = waiting.loc[i,'node_id']
    train_num = train_interval(time=7, node_id=node_id,time_delta=3)
    waiting.loc[i,'train_num']= train_num

In [479]:
waiting[waiting['train_num']==0].head()

,stop_id,train,stop_name,stop_lat,stop_lon,node_id,master_node_id,train_num
38,121,2,86 St,40.788644,-73.976218,121_2,86 St_0,0
39,122,2,79 St,40.783934,-73.979917,122_2,79 St_0,0
41,124,2,66 St - Lincoln Center,40.773440,-73.982209,124_2,66 St - Lincoln Center_0,0
42,125,2,59 St - Columbus Circle,40.768247,-73.981929,125_2,59 St - Columbus Circle_0,0
43,126,2,50 St,40.761728,-73.983849,126_2,50 St_0,0


These stations have no train going through. The main reason is the node_id take all the late night schedule into consideration. But some trains don't stop at those stations during rush hour/ daytime schedule.

In [480]:
waiting = waiting[waiting['train_num']!=0].reset_index(drop=True)

In [483]:
waiting['interval'] = (3*60/waiting['train_num'])*60

In [487]:
waiting.head()

,stop_id,train,stop_name,stop_lat,stop_lon,node_id,master_node_id,train_num,interval
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583,101_1,Van Cortlandt Park - 242 St_0,29,372.414
1,103,1,238 St,40.884667,-73.900870,103_1,238 St_0,33,327.273
2,104,1,231 St,40.878856,-73.904834,104_1,231 St_0,33,327.273
3,106,1,Marble Hill - 225 St,40.874561,-73.909831,106_1,Marble Hill - 225 St_0,33.5,322.388
4,107,1,215 St,40.869444,-73.915279,107_1,215 St_0,34.5,313.043


In [489]:
waiting['waiting'] =waiting['interval']/2

In [490]:
waiting.head()

,stop_id,train,stop_name,stop_lat,stop_lon,node_id,master_node_id,train_num,interval,waiting
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583,101_1,Van Cortlandt Park - 242 St_0,29,372.414,186.207
1,103,1,238 St,40.884667,-73.900870,103_1,238 St_0,33,327.273,163.636
2,104,1,231 St,40.878856,-73.904834,104_1,231 St_0,33,327.273,163.636
3,106,1,Marble Hill - 225 St,40.874561,-73.909831,106_1,Marble Hill - 225 St_0,33.5,322.388,161.194
4,107,1,215 St,40.869444,-73.915279,107_1,215 St_0,34.5,313.043,156.522


In [491]:
waiting.to_csv('waiting')